# Watermark Embedding and Extraction using SVD in RGB Channels

This notebook covers the embedding and extraction of watermarks in images using Singular Value Decomposition (SVD) in RGB Channels. It includes loading images, resizing, converting to grayscale, performing SVD operations, embedding the watermark, and extracting it.

## Embedding Watermark


This script demonstrates how to embed a watermark image into a host image using Singular Value Decomposition (SVD). The watermark is embedded into each RGB channel of the host image separately. The embedding process is followed by a watermark extraction process to retrieve the watermark from the watermarked image.

In [1]:
from tqdm import tqdm
from PIL import Image
import numpy as np
import cv2

### Load the host and watermark images


The host image is the primary image where the watermark will be embedded. The watermark image is the image that will be hidden within the host image.

In [2]:
hr_image = Image.open("./images/inputs/host_image.jpg")
wm_image = Image.open("./images/inputs/watermark_image.jpg")

### Resize and Convert Images to RGB

The watermark image is resized to match the size of the host image. Both images are converted to the RGB color mode if they aren't already.

In [3]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)

# Convert images to RGB (if not already in that mode)
hr_image = hr_image.convert("RGB")
wm_image_resized = wm_image_resized.convert("RGB")

# Split images into R, G, B channels
hr_r, hr_g, hr_b = hr_image.split()
wm_r, wm_g, wm_b = wm_image_resized.split()

### Convert Channels to Numpy Arrays

The image channels are converted to numpy arrays to perform mathematical operations.

In [4]:
# Convert channels to numpy arrays
hr_r = np.array(hr_r)
hr_g = np.array(hr_g)
hr_b = np.array(hr_b)
# hr_r, hr_g, hr_b

In [5]:
wm_r = np.array(wm_r)
wm_g = np.array(wm_g)
wm_b = np.array(wm_b)
# wm_r, wm_g, wm_b

### Function to Apply SVD and Modify Singular Values


This function applies Singular Value Decomposition (SVD) to both the host and watermark channels and then modifies the singular values of the host image based on the singular values of the watermark.

In [6]:
def embed_watermark(hr_channel, wm_channel, alpha=0.6,channel_name=None):
    """
    Embeds a watermark into a host image channel using SVD.

    Args:
        hr_channel (numpy array): Host image channel.
        wm_channel (numpy array): Watermark image channel.
        alpha (float): Scaling factor for watermark embedding.

    Returns:
        hr_channel_mod (numpy array): Watermarked host image channel.
    """
    with tqdm(total=100, desc=f"Embedding Watermark in a {channel_name} Channel", bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
        U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel)
        pbar.update(40)

        U_wm, sigma_wm, V_wm = np.linalg.svd(wm_channel)
        pbar.update(40)

        # Modify the singular values by adding scaled watermark singular values
        sigma_hr_mod = sigma_hr + alpha * sigma_wm
        hr_channel_mod = np.dot(U_hr[:, :sigma_hr_mod.shape[0]], np.dot(
            np.diag(sigma_hr_mod), V_hr[:sigma_hr_mod.shape[0], :]))
        pbar.update(20)

    return hr_channel_mod

### Embed Watermark in Each RGB Channel

The watermark is embedded separately in each of the R, G, and B channels of the host image.

In [7]:
hr_r_mod = embed_watermark(hr_r, wm_r,channel_name="Red")
hr_g_mod = embed_watermark(hr_g, wm_g,channel_name="Green")
hr_b_mod = embed_watermark(hr_b, wm_b,channel_name="Blue")

Embedding Watermark in a Red Channel: 100%|██████████ [ time left: 00:00 ]
Embedding Watermark in a Green Channel: 100%|██████████ [ time left: 00:00 ]
Embedding Watermark in a Blue Channel: 100%|██████████ [ time left: 00:00 ]


### Normalize the Modified Channels

The modified channels are normalized to the range [0, 255] and converted to uint8 format.

In [21]:
# Normalize and convert to uint8
hr_r_mod = cv2.normalize(hr_r_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
hr_g_mod = cv2.normalize(hr_g_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
hr_b_mod = cv2.normalize(hr_b_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
# hr_r_mod, hr_g_mod, hr_b_mod

### Merge Modified Channels to Form Watermarked Image and Save the Watermarked Image

The normalized R, G, and B channels are merged to form the final watermarked image and Display the watermarked image and save it as `watermarked_image_rgb.png`.


In [9]:
# Merge modified channels back into an RGB image
watermarked_image = Image.merge("RGB", (Image.fromarray(
    hr_r_mod), Image.fromarray(hr_g_mod), Image.fromarray(hr_b_mod)))
watermarked_image.save('./images/outputs/watermarked_image_rgb.png')
watermarked_image.show()

## Extracting Watermark

The watermark is extracted from the watermarked image by reversing the embedding process.

In [10]:
from PIL import Image
import numpy as np
import cv2

### Load the Host, Watermark, and Watermarked Images

The original host image, the watermark image, and the watermarked image are loaded and split into their respective RGB channels.

In [11]:
hr_image = Image.open("./images/inputs/host_image.jpg")
wm_image = Image.open("./images/inputs/watermark_image.jpg")
watermarked_image = Image.open("./images/outputs/watermarked_image_rgb.png")

### Resize and Convert Images to RGB

The watermark image and watermarked image is resized to match the size of the host image. Both images are converted to the RGB color mode if they aren't already.

In [12]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)
watermarked_image_resized = watermarked_image.resize(hr_image.size)

# Convert images to RGB (if not already in that mode)
hr_image = hr_image.convert("RGB")
wm_image_resized = wm_image_resized.convert("RGB")
watermarked_image_resized = watermarked_image_resized.convert("RGB")

# Split images into R, G, B channels
hr_r, hr_g, hr_b = hr_image.split()
wm_r, wm_g, wm_b = wm_image_resized.split()
hr_r_mod, hr_g_mod, hr_b_mod = watermarked_image_resized.split()

### Convert Channels to Numpy Arrays

The RGB channels of the host, watermark, and watermarked images are converted to numpy arrays for mathematical operations.

In [13]:
# Convert channels to numpy arrays
hr_r = np.array(hr_r)
hr_g = np.array(hr_g)
hr_b = np.array(hr_b)
# hr_r, hr_g, hr_b

In [14]:
wm_r = np.array(wm_r)
wm_g = np.array(wm_g)
wm_b = np.array(wm_b)
# wm_r, wm_g, wm_b

In [15]:
hr_r_mod = np.array(hr_r_mod)
hr_g_mod = np.array(hr_g_mod)
hr_b_mod = np.array(hr_b_mod)
# hr_r_mod, hr_g_mod, hr_b_mod

### Function to Extract Watermark from Each Channel

The function extracts the watermark from the watermarked image by performing SVD on both the watermarked and original host image channels and calculating the difference in singular values.

In [16]:
def extract_watermark(hr_mod_channel, hr_channel, wr_channel, alpha=0.6, channel_name=None):
    """
    Extracts the watermark from a watermarked channel.

    Args:
        hr_mod_channel (numpy array): Watermarked host image channel.
        hr_channel (numpy array): Original host image channel.
        wr_channel (numpy array): Watermark image channel.
        alpha (float): Scaling factor used during embedding.

    Returns:
        wm_channel_mod (numpy array): Extracted watermark channel.
    """
    with tqdm(total=100, desc=f"Extracting Watermark from a {channel_name} Channel", bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
        U_hr_mod, sigma_hr_mod, V_hr_mod = np.linalg.svd(hr_mod_channel, full_matrices=False)
        pbar.update(30)

        U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel, full_matrices=False)
        pbar.update(30)

        U_wm, sigma_wm, V_wm = np.linalg.svd(wr_channel, full_matrices=False)
        pbar.update(30)

        # Calculate the modified watermark channel
        sigma_wm_mod = (sigma_hr_mod - sigma_hr) / alpha
        wm_channel_mod = np.dot(U_wm, np.dot(np.diag(sigma_wm_mod), V_wm))
        pbar.update(10)

    return wm_channel_mod

### Extract Watermark from Each RGB Channel

The watermark is extracted separately from the R, G, and B channels of the watermarked image.

In [17]:
wm_r_mod = extract_watermark(hr_r_mod, hr_r, wm_r,channel_name="Red")
wm_g_mod = extract_watermark(hr_g_mod, hr_g, wm_g,channel_name="Green")
wm_b_mod = extract_watermark(hr_b_mod, hr_b, wm_b,channel_name="Blue")

Extracting Watermark from a Red Channel: 100%|██████████ [ time left: 00:00 ]
Extracting Watermark from a Green Channel: 100%|██████████ [ time left: 00:00 ]
Extracting Watermark from a Blue Channel: 100%|██████████ [ time left: 00:00 ]


### Normalize and Merge Extracted Channels

The extracted R, G, and B channels are normalized and merged to form the final extracted watermark image.

In [20]:
# Normalize and convert to uint8
wm_r_mod = cv2.normalize(wm_r_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
wm_g_mod = cv2.normalize(wm_g_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
wm_b_mod = cv2.normalize(wm_b_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
# wm_r_mod, wm_g_mod, wm_b_mod

In [19]:
# Merge extracted channels back into an RGB image
extracted_watermark = Image.merge("RGB", (Image.fromarray(
    wm_r_mod), Image.fromarray(wm_g_mod), Image.fromarray(wm_b_mod)))
extracted_watermark.save('./images/outputs/extracted_watermark_image_rgb.png')
extracted_watermark.show()